In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import hvplot.pandas

In [ ]:
df=pd.read_csv('../data/Transect_20220302_Fabian.csv')
df['EC'] = df['Sonde1_SpCond']

In [ ]:
df.columns

In [ ]:
df.hvplot.points('Longitude','Latitude',geo=True,tiles='CartoLight', s='Sonde1_SpCond')

In [ ]:
import holoviews as hv

from holoviews import dim,opts

import geoviews as gv

In [ ]:
hv.element.tile_sources.get('CartoLight')()

In [ ]:
pts = gv.Points(df, kdims=['Longitude','Latitude'], vdims=['Sonde1_SpCond']).opts(opts.Points(color='Sonde1_SpCond'))

In [ ]:
map = hv.element.tiles.CartoLight()*pts.opts(cmap='rainbow4', colorbar=True)
map.opts(frame_width=500)

In [ ]:
# Attemtp to show ec as vector field. failed
#dfa = df.copy()

#import math
#dfa['angle']=math.pi/2

#ec_vectors = dfa.hvplot.vectorfield(x='Longitude',y='Latitude',angle='angle',mag='Sonde1_SpCond',geo=True, hover=[]).opts(magnitude='Sonde1_SpCond', frame_width=800, frame_height=1000)
#ec_vectors = ec_vectors.opts(opts.VectorField(alpha=0.85,color='black',pivot='tail',line_width=2,line_cap='round'))

#ec_vectors

In [ ]:
import numpy as np
def create_scaled_points_legend(ns, max_point_size, min_point_size):
    #x,y,sz_arr,labels
    data = {
        'x': np.zeros(ns.maxTicks),
        'y':  np.arange(1,0,-1./ns.maxTicks),
        'size': np.array([round((ns.tickSpacing*i)/(ns.niceMax-ns.niceMin)*max_point_size+min_point_size) for i in range(ns.maxTicks)]),
        'labels':[str(round(a*ns.tickSpacing+ns.niceMin)) for a in range(ns.maxTicks)]
    }
    #labels = hv.Labels({'x', 'y'): (data['x'], data['y']), 'text': [1, 5, 20]}, ['x', 'y'], 'text')
    values = np.array(data['size'])+18
    lbls = hv.Labels(data, kdims=['x','y'], vdims='labels').opts(opts.Labels(text_align='left',text_baseline='middle',xoffset=0.4))

    #opts.defaults(opts.Points(size=8))

    pts = hv.Points(data, kdims=['x','y'] , vdims=['size']).opts(size='size', ylim=(0,1.1)).opts(opts.Points(fill_alpha=0))

    return (lbls*pts).opts(xaxis=None, yaxis=None, toolbar=None, frame_height=150, frame_width=100)

In [ ]:
from transect_viz.nice_scale import NiceScale
vals=df['EC'].values
ns=NiceScale(np.min(vals),np.max(vals))
min_point_size=1
max_point_size=25
points_dim=(dim('Sonde1_SpCond')-ns.niceMin)/(ns.niceMax-ns.niceMin)*max_point_size+min_point_size

In [ ]:
spts = gv.Points(df, kdims=['Longitude','Latitude'], vdims=['Sonde1_SpCond']).opts(opts.Points(size=points_dim))

In [ ]:
create_scaled_points_legend(ns,25,1).opts(frame_height=150)

In [ ]:
smap = hv.element.tiles.CartoLight()*spts
smap.opts(frame_width=500)

In [ ]:
scpts = gv.Points(df, kdims=['Longitude','Latitude'], vdims=['Sonde1_SpCond','DateTime']).opts(opts.Points(tools=['hover'], color='Sonde1_SpCond', size=dim('Sonde1_SpCond')/25-18, alpha=0.5, cmap='fire'))

In [ ]:
scmap = hv.element.tiles.CartoLight()*scpts.opts(cmap='ColorBlind', colorbar=True)
scmap = scmap.opts(frame_width=500, title='Color and Point sizes scaled by Sonde1 Specific Conductivity', )
scmap

In [ ]:
pts_legend = create_scaled_points_legend(ns, max_point_size=25, min_point_size=1)
scmap + pts_legend

In [ ]:
hv.save(scmap.opts(frame_width=1200), 'sonde1_sp_color_size.html')

# CDEC stations with flow

In [ ]:
sdate,edate=df.DateTime.agg(['min','max'])
start_date = pd.to_datetime(sdate).floor('1D')-pd.Timedelta('2D')
end_date=pd.to_datetime(edate).ceil('1D')+pd.Timedelta('2D')

In [ ]:
from transect_viz import cdec

In [ ]:
cr=cdec.Reader()

In [ ]:
cdec_flow_stations=['ORM','ODM', 'OLD','GLE','GLC']

In [ ]:
print(' Getting FLOW data from CDEC for ',cdec_flow_stations, ' from ', start_date, ' to ',end_date)
raw_cdec_flows = [cr.read_station_data(station_id,20,'E', start_date, end_date) for station_id in cdec_flow_stations]

In [ ]:
def get_lon_lat(station_id):
    cr=cdec.Reader()
    dfmeta = cr.read_station_meta_info(station_id)[0]
    dfmeta = dfmeta.reset_index().set_index('Value')
    degree_sign = u'\N{DEGREE SIGN}'
    return float(dfmeta.loc['Longitude'][0].replace(degree_sign,'')), float(dfmeta.loc['Latitude'][0].replace(degree_sign,''))

In [ ]:
cdec_flow_stations

In [ ]:
dfs = pd.DataFrame.from_records({sid:get_lon_lat(sid) for sid in cdec_flow_stations}).T
dfs=dfs.reset_index()
dfs.columns=['Station ID','Longitude','Latitude']
dfs

In [ ]:
station_pts = dfs.hvplot.points(x='Longitude',y='Latitude',geo=True,hover_cols=['Station ID']).opts(frame_width=800)

In [ ]:
station_labels = dfs.hvplot.labels(x='Longitude',y='Latitude',text='Station ID', geo=True, text_align='left_top').opts(
    opts.Labels(xoffset=100, yoffset=300))

In [ ]:
stations_map = station_pts*station_labels

In [ ]:
scmap*stations_map

In [ ]:
cdec_flows = [pd.DataFrame(dflow.VALUE).rename(columns={'VALUE':dflow.STATION_ID.iloc[0]}) for dflow in raw_cdec_flows]

In [ ]:
sdate,edate

In [ ]:
flow_around_time = hv.Overlay([dflow.hvplot(label=dflow.columns[0]) for dflow in cdec_flows]).opts(
    ylabel='FLOW (cfs)', title='Flow at locations',
    xlim=(pd.to_datetime(sdate)-pd.Timedelta('1D'), pd.to_datetime(edate)+pd.Timedelta('1D')))
flow_around_time

In [ ]:
collection_timespan = hv.VSpan(x1=pd.to_datetime(sdate),x2=pd.to_datetime(edate))

In [ ]:
flow_around_time*collection_timespan

In [ ]:
from vtools.functions.filter import godin

In [ ]:
tidal_cdec_flows = [godin(dflow.resample('1H').mean()) for dflow in cdec_flows]

In [ ]:
hv.Overlay([dflow.hvplot(label=dflow.columns[0]) for dflow in tidal_cdec_flows]).opts(xlim=(pd.to_datetime(sdate),pd.to_datetime(edate))
                                                         , title = 'Tidal Flow at locations')

In [ ]:
tidal_flow_around_time = hv.Overlay([dflow.hvplot(label=dflow.columns[0]) for dflow in tidal_cdec_flows]).opts(title = 'Tidal Flow at locations')

In [ ]:
tidal_flow_around_time*collection_timespan

In [ ]:
print(' Getting VELOCITY data from CDEC for ',cdec_flow_stations, ' from ', start_date, ' to ',end_date)
raw_cdec_vels = [cr.read_station_data(station_id,21,'E', start_date, end_date) for station_id in cdec_flow_stations]

In [ ]:
cdec_vels = [pd.DataFrame(dvel.VALUE).rename(columns={'VALUE':dvel.STATION_ID.iloc[0]}) for dvel in raw_cdec_vels]

In [ ]:
vel_around_time = hv.Overlay([dvel.hvplot(label=dvel.columns[0]) for dvel in cdec_vels]).opts(
    ylabel='vel (ft/sec)', title='vel at locations',
    xlim=(pd.to_datetime(sdate)-pd.Timedelta('1D'), pd.to_datetime(edate)+pd.Timedelta('1D')))
vel_around_time*collection_timespan

In [ ]:
travel_distances=[v.cumsum()*3600/5280.0 for v in cdec_vels]

In [ ]:
hv.Overlay([d.hvplot(label=d.columns[0]) for d in travel_distances])*collection_timespan

In [ ]:
dfvel = pd.DataFrame([v[sdate:edate].mean()[0] for v in cdec_vels], index=[v.columns[0] for v in cdec_vels])
dfvel.columns=['mean velocity']
dfvel

In [ ]:
dfvelgis = dfs.set_index('Station ID').join(dfvel)

In [ ]:
import math

In [ ]:
dfvelgis['angle']=math.pi
dfvelgis.loc['ODM','angle'] = 120/180*math.pi
dfvelgis.loc['ORM','angle'] = 120/180*math.pi
dfvelgis.loc['OLD','angle'] = 220/180*math.pi


In [ ]:
dfvelgis

In [ ]:
vel_vectors=dfvelgis.hvplot.vectorfield(x='Longitude',y='Latitude',angle='angle',mag='mean velocity',geo=True,hover=[]).opts(magnitude='mean velocity')
vel_vectors = vel_vectors.opts(opts.VectorField(alpha=0.85,color='blue',pivot='tail',line_width=10,line_cap='round'))

In [ ]:
dfvelgis['vel_label']=dfvelgis['mean velocity'].map('{:,.2f} ft/s'.format)

In [ ]:
vel_labels = dfvelgis.hvplot.labels(x='Longitude',y='Latitude',text='vel_label',geo=True).opts(opts.Labels(text_align='top', xoffset=400, yoffset=800))

In [ ]:
scmap*stations_map*vel_vectors*vel_labels